# Prepare data

In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
from aimsonnx import model_to_onnx, _get_metadata
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [67]:
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Scikit-Learn

In [68]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from skl2onnx.common.data_types import FloatTensorType


### Instantiate model

In [69]:
model_sklearn = MLPClassifier(hidden_layer_sizes=(5,5))
model_sklearn.fit(X_train, y_train)

C:\Users\heinr\anaconda3\envs\aimodelshare\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(5, 5))

### Transform to ONNX

In [70]:
# specify intial types of predictors
initial_type = [('float_input', FloatTensorType([None, 4]))]

# transform sklearn model to ONNX
onnx_model_sklearn = model_to_onnx(model_sklearn, framework='sklearn', 
                                   initial_types=initial_type,
                                   transfer_learning=False,
                                   deep_learning=True,
                                   task_type='classification')

list index out of range


### Check metadata

In [71]:
_get_metadata(onnx_model_sklearn)

{'model_metadata': '{\'model_id\': None, \'data_id\': None, \'preprocessor_id\': None, \'ml_framework\': \'sklearn\', \'model_type\': \'MLPClassifier(hidden_layer_sizes=(5, 5))\', \'transfer_learning\': False, \'deep_learning\': True, \'task_type\': \'classification\', \'target_distribution\': None, \'input_type\': None, \'input_shape\': None, \'input_dtypes\': None, \'input_distribution\': None, \'model_config\': "{\'activation\': \'relu\', \'alpha\': 0.0001, \'batch_size\': \'auto\', \'beta_1\': 0.9, \'beta_2\': 0.999, \'early_stopping\': False, \'epsilon\': 1e-08, \'hidden_layer_sizes\': (5, 5), \'learning_rate\': \'constant\', \'learning_rate_init\': 0.001, \'max_fun\': 15000, \'max_iter\': 200, \'momentum\': 0.9, \'n_iter_no_change\': 10, \'nesterovs_momentum\': True, \'power_t\': 0.5, \'random_state\': None, \'shuffle\': True, \'solver\': \'adam\', \'tol\': 0.0001, \'validation_fraction\': 0.1, \'verbose\': False, \'warm_start\': False}", \'model_state\': None, \'model_architectu

# Keras

In [72]:
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

### Instantiate model

In [73]:
model_keras = Sequential()
model_keras.add(Input(shape=(4,)))
model_keras.add(Dense(units=5, activation='relu'))
model_keras.add(Dense(units=5, activation='relu'))
model_keras.add(Dense(units=3, activation='softmax'))

model_keras.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

In [74]:
model_keras.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 5)                 25        
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 30        
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 18        
Total params: 73
Trainable params: 73
Non-trainable params: 0
_________________________________________________________________


In [75]:
model_keras.fit(X_train, to_categorical(y_train, 3))

4/4 [==============================] - 0s 499us/step - loss: 2.6581 - accuracy: 0.3125


### Transform to ONNX

In [76]:
# drop in replacement 
onnx_model_keras = model_to_onnx(model_keras, framework='keras',
                                 transfer_learning=False,
                                 deep_learning=True,
                                 task_type='classification')

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 13 -> 9
The maximum opset needed by this model is only 9.


### Check metadata

In [77]:
_get_metadata(onnx_model_keras)

{'model_metadata': '{\'model_id\': None, \'data_id\': None, \'preprocessor_id\': None, \'ml_framework\': \'keras\', \'model_type\': \'Sequential\', \'transfer_learning\': False, \'deep_learning\': True, \'task_type\': \'classification\', \'target_distribution\': None, \'input_type\': None, \'input_shape\': None, \'input_dtypes\': None, \'input_distribution\': None, \'model_config\': "{\'name\': \'sequential_3\', \'layers\': [{\'class_name\': \'InputLayer\', \'config\': {\'batch_input_shape\': (None, 4), \'dtype\': \'float32\', \'sparse\': False, \'ragged\': False, \'name\': \'input_4\'}}, {\'class_name\': \'Dense\', \'config\': {\'name\': \'dense_9\', \'trainable\': True, \'dtype\': \'float32\', \'units\': 5, \'activation\': \'relu\', \'use_bias\': True, \'kernel_initializer\': {\'class_name\': \'GlorotUniform\', \'config\': {\'seed\': None}}, \'bias_initializer\': {\'class_name\': \'Zeros\', \'config\': {}}, \'kernel_regularizer\': None, \'bias_regularizer\': None, \'activity_regulari

# Pytorch

In [78]:
from torch import nn
import torch

### Instantiate model

In [79]:
model_pytorch = nn.Sequential(nn.Linear(4, 5),
                      nn.ReLU(),
                      nn.Linear(5, 5),
                      nn.ReLU(),
                      nn.Linear(5, 3),
                      nn.Softmax(dim=1))

### Transform to ONNX

In [80]:
dummy_input = torch.randn(1, 4)
onnx_model_pytorch = model_to_onnx(model_pytorch, framework='pytorch',
                                   model_input=dummy_input, 
                                   transfer_learning=False,
                                   deep_learning=True,
                                   task_type='classification')

### Check metadata

In [81]:
_get_metadata(onnx_model_pytorch)

{'model_metadata': '{\'model_id\': None, \'data_id\': None, \'preprocessor_id\': None, \'ml_framework\': \'pytorch\', \'model_type\': \'Sequential()\', \'transfer_learning\': False, \'deep_learning\': True, \'task_type\': \'classification\', \'target_distribution\': None, \'input_type\': None, \'input_shape\': None, \'input_dtypes\': None, \'input_distribution\': None, \'model_config\': "{\'training\': True, \'_parameters\': OrderedDict(), \'_buffers\': OrderedDict(), \'_non_persistent_buffers_set\': set(), \'_backward_hooks\': OrderedDict(), \'_forward_hooks\': OrderedDict(), \'_forward_pre_hooks\': OrderedDict(), \'_state_dict_hooks\': OrderedDict(), \'_load_state_dict_pre_hooks\': OrderedDict(), \'_modules\': OrderedDict([(\'0\', Linear(in_features=4, out_features=5, bias=True)), (\'1\', ReLU()), (\'2\', Linear(in_features=5, out_features=5, bias=True)), (\'3\', ReLU()), (\'4\', Linear(in_features=5, out_features=3, bias=True)), (\'5\', Softmax(dim=1))])}", \'model_state\': "Ordered